In [337]:
import sys
import json
from tqdm import tqdm
import re
import pandas as pd
from collections import defaultdict
from libindic import inexactsearch
import pandas as pd
import pickle as pkl
from editdistance import eval as ed
from fuzzywuzzy import fuzz
import numpy as np
from random import shuffle

In [384]:
def get_score(translated_infobox , actual_infobox , debug = False):
    inst = inexactsearch.InexactSearch()
    markings = defaultdict(list)
    all_keys = list(translated_infobox.keys()) + list(actual_infobox.keys())
    translated_infobox = { key.strip() : val for key , val in translated_infobox.items()}
    actual_infobox = { key.strip() : val for key , val in actual_infobox.items()}

    for key in translated_infobox.keys():
        for key2 in actual_infobox.keys():
            if debug and fuzz.token_sort_ratio(key , key2) > 80:
                print(key , key2)
                print(fuzz.token_sort_ratio(key , key2))
            
            if fuzz.token_sort_ratio(key , key2) > 80:
                if not translated_infobox[key]:
                    markings['S'].append(key)
                    continue 
                elif not actual_infobox[key2]:
                    continue
                try :
                    val = inst.compare(translated_infobox[key] , actual_infobox[key2])
                except :
                    val = 0
                if val > 0.6 :
                    markings['C'].append(key)
                else :
                    markings['S'].append(key)
                break
    for key2 in actual_infobox.keys():
        for key in translated_infobox.keys():
            if fuzz.token_sort_ratio(key , key2) > 80:
                break
        else : 
            markings['D'].append(key2)
    if debug :
        print(translated_infobox)
        print(actual_infobox)
    if len(markings['C']) + len(markings['S']) != 0:
        precision = len(markings['C']) / (len(markings['C']) + len(markings['S']))
    else : precision = 0
    if (len(markings['C']) + len(markings['S']) + len(markings['D'])) != 0:
        recall = len(markings['C']) / (len(markings['C']) + len(markings['S']) + len(markings['D']))
    else : recall = 0
    return [precision , recall]

In [448]:
def get_results(data):
    overall_score = []
    for ind , entry in enumerate(data) :
        cur_score = []
        for key , val in entry.items():
            for i in range(3):
                if not val[-1] : continue
                cur_score.append(get_score(val[2 - i] , val[-1] , debug = False))
        if not cur_score : continue
        overall_score.append(np.array(sum(cur_score , [])))
    df = pd.DataFrame(overall_score , columns=['M1-Precision','M1-Recall','M2-Precision','M2-Recall','Baseline-Precision','Baseline-Recall'])
    return df

In [477]:
data = []
with open('people_eval_records.jsonl') as f:
    data = [ json.loads(i) for i in f.readlines()]
print("Number of records for people :" , len(data))
df = get_results(data)
display(df.mean())

Number of records for people : 700


M1-Precision          0.324483
M1-Recall             0.064490
M2-Precision          0.422719
M2-Recall             0.213753
Baseline-Precision    0.264609
Baseline-Recall       0.056198
dtype: float64

In [478]:
data = []
with open('places_eval_records.jsonl') as f:
    data = [ json.loads(i) for i in f.readlines()]
print("Number of records for places :" , len(data))
df = get_results(data)
display(df.mean())

Number of records for places : 400


M1-Precision          0.480417
M1-Recall             0.019363
M2-Precision          0.169016
M2-Recall             0.044931
Baseline-Precision    0.356667
Baseline-Recall       0.013532
dtype: float64